In [1]:
from __future__ import print_function
from Bio.PDB import *
import os
import numpy as np
import collections
import pandas as pd
from scipy.spatial import distance
from pygsp import graphs, features
import networkx as nx
import matplotlib.pyplot as plt
import pickle

In [2]:
amino_lookup = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N',
     'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W',
     'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M','CCS':'C','AC5':'L'}
amino_molecular_mass = {'A': 89.09404, 'R': 174.20274, 'N': 132.11904, 'D': 133.10384, 'C': 121.15404,
                        'Q': 146.14594, 'E': 147.13074, 'G': 75.06714, 'H': 155.15634, 'I': 131.17464,
                        'L': 131.17464, 'K': 146.18934, 'M': 149.20784, 'F': 165.19184, 'P': 115.13194,
                        'S': 105.09344, 'T': 119.12034, 'W': 204.22844, 'Y': 181.19124, 'V': 117.14784}
amino_hydrophobicity = {'A': 1.8, 'R': -4.5, 'N': -3.5, 'D': -3.5, 'C': 2.5,
                        'Q': -3.5, 'E': -3.5, 'G': -0.4, 'H': -3.2, 'I': 4.5,
                        'L': 3.8, 'K': -3.9, 'M': 1.9, 'F': 2.8, 'P': -1.6,
                        'S': -0.8, 'T': -0.7, 'W': -0.9, 'Y': -1.3, 'V': 4.2}

amino_bulkiness = {'A':11.5, 'D':11.68,'C':13.46,'E':13.57,'F':19.8,'G':3.4,'H':13.67,'I':21.4,'K':15.71,'L':21.4,
                   'M':16.25,'N':12.82,'P':17.43,'Q':14.45,'R':14.28,'S':9.47,'T':15.77,'V':21.57,'W':21.61,'Y':18.03}

amino_polarity = {'A':0, 'D':49.7,'C':1.48,'E':49.9,'F':0.35,'G':0,'H':51.6,'I':0.1,'K':49.5,'L':0.13,
                   'M':1.43,'N':3.38,'P':1.58,'Q':3.53,'R':52,'S':1.67,'T':1.66,'V':0.13,'W':2.1,'Y':1.61}

amino_turn_tendency = {'A':0.66, 'D':1.46,'C':1.19,'E':0.74,'F':0.6,'G':1.56,'H':0.95,'I':0.47,'K':1.01,'L':0.59,
                   'M':0.6,'N':1.56,'P':1.52,'Q':0.98,'R':0.95,'S':1.43,'T':0.96,'V':0.5,'W':0.96,'Y':1.14}

amino_coil_tendency = {'A':0.71, 'D':1.21,'C':1.19,'E':0.84,'F':0.71,'G':1.52,'H':1.07,'I':0.66,'K':0.99,'L':0.69,
                   'M':0.59,'N':1.37,'P':1.61,'Q':0.87,'R':1.07,'S':1.34,'T':1.08,'V':0.63,'W':0.76,'Y':1.07}

amino_flexibility = {'A':0, 'D':2,'C':1,'E':3,'F':2,'G':0,'H':2,'I':2,'K':4,'L':2,
                   'M':3,'N':2,'P':0,'Q':3,'R':5,'S':1,'T':1,'V':1,'W':2,'Y':2}

amino_partial_specific_volume = {'A':60.46, 'D':73.83,'C':67.7,'E':85.88,'F':121.48,'G':43.25,'H':98.79,
                                 'I':107.72,'K':108.5,
                                 'L':107.75,'M':105.35,'N':78.01,'P':82.83,'Q':93.9,
                                 'R':127.34,'S':60.62,'T':76.83,'V':90.78,'W':143.91,'Y':123.6}

amino_compressibility = {'A':-25.5, 'D':-33.12,'C':-32.82,'E':-36.17,'F':-34.54,'G':-27,'H':-31.84,
                        'I':-31.78,'K':-32.4,
                        'L':-31.78,'M':-31.18,'N':-30.9,'P':-23.25,'Q':-32.6,
                        'R':-26.62,'S':-29.88,'T':-31.23,'V':-30.62,'W':-30.24,'Y':-35.01}

amino_refractive_index = {'A':14.34, 'D':12,'C':35.77,'E':17.26,'F':29.4,'G':0,'H':21.81,
                        'I':19.06,'K':21.29,
                        'L':18.78,'M':21.64,'N':13.28,'P':10.93,'Q':17.56,
                        'R':26.66,'S':6.35,'T':11.01,'V':13.92,'W':42.53,'Y':31.55}



In [22]:
def crawl_pdb(path):
    '''This funciton reads pdb files and stores their distance matrix and sequence'''
    parser = PDBParser()
    pdb_files = sorted(os.listdir(path))
    pdbinfo_dict = dict()
    for pdb in pdb_files:
        info = dict()
        info[id] = pdb
        structure = parser.get_structure('pdb_file', path  + pdb )
        coordinates = []
        labels = list()
        for model in structure:
            for chain in model:
                for residue in chain:
                    try:
                        if residue.get_resname() in amino_lookup:
                            coordinates.append(residue['CA'].get_coord())
                            labels.append(residue.get_resname())
                    except KeyError:
                        pass
                break  ## working on chain id A only
            break      ## Working on model id 0 only
        coords = np.asmatrix(coordinates)
        distance_matrix = distance.squareform(distance.pdist(coords))
        info['coords'] = coords
        info['distance_matrix'] = distance_matrix
        #print(np.unique(labels))
        info['sequence'] = ''.join([amino_lookup[s] for s in labels if s in amino_lookup])
        pdbinfo_dict[pdb] = info
    return pdbinfo_dict


def get_graph(distance_matrix, network_type, rig_cutoff=7.3, lin_cutoff=12):
    distance_matrix[distance_matrix >= rig_cutoff] = 0
    if network_type == 'rig-boolean':
        distance_matrix[distance_matrix > 0] = 1
    elif network_type == 'weighted-rig':
        for i in range(np.shape(distance_matrix)[0]):
            for j in range(np.shape(distance_matrix)[1]):
                if distance_matrix[i, j] > 0:
                    distance_matrix[i, j] = abs(j - i)
    elif network_type == 'weighted-lin':
        for i in range(np.shape(distance_matrix)[0]):
            for j in range(np.shape(distance_matrix)[1]):
                if distance_matrix[i, j] > 0:
                    if abs(i - j) >= lin_cutoff or abs(i - j) == 1:
                        distance_matrix[i, j] = abs(i - j)
                    else:
                        distance_matrix[i, j] = 0
    elif network_type == 'lin':
        for i in range(np.shape(distance_matrix)[0]):
            for j in range(np.shape(distance_matrix)[1]):
                if distance_matrix[i, j] > 0:
                    if abs(i - j) >= lin_cutoff or abs(i - j) == 1:
                        distance_matrix[i, j] = 1
                    else:
                        distance_matrix[i, j] = 0
    else:
        print('Invalid Choice! ' + network_type)
        return None
    G = graphs.Graph(distance_matrix, lap_type='normalized')
    G.compute_fourier_basis()
    return G


def get_signal(G, seq, signal):
    if signal == 'molecular_weight':
        s = np.asarray([amino_molecular_mass[aa] for aa in seq])
    elif signal == 'hydrophobicity':
        s = np.asarray([amino_hydrophobicity[aa] for aa in seq])
    elif signal == 'node_degree':
        s = G.d
    elif signal == 'node_weighted_degree':
        adj = G.W.todense()
        s = np.ravel(adj.sum(axis=0)) / 2
    elif signal == 'avg_adj_degree':
        s = features.compute_avg_adj_deg(G)
        s = np.ravel(s)
    elif signal == 'clustering_coeff':
        N = nx.from_scipy_sparse_matrix(G.W)
        s = nx.clustering(N)
        s = np.asarray(list(s.values()))
    elif signal == 'aaalpha_helix':
        s = eng.aaalpha_helixfasman(seq)
        s = np.array(s._data)
    elif signal == 'residue_count':
        residue_counts = collections.Counter(seq)
        s = np.asarray([residue_counts[s] for s in seq])
    else:
        print ('Invalid Choice! ' + signal)
    return s


def get_filtered_signal(G, signal, cutoff):
    signal_hat = G.gft(signal)
    signal_filtered_hat = np.zeros_like(signal_hat)
    signal_filtered_hat[G.e < G.lmax * cutoff] = signal_hat[G.e < G.lmax * cutoff]
    signal_filtered = G.igft(signal_filtered_hat)
    return signal_filtered

In [23]:
prion_path = '/home/cellsearch/cellatlassearch_shreya/graphwavelet+cnn/new_pdb/'
pdbinfo_dict = crawl_pdb(prion_path)

/home/cellsearch/py_36_env/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 2912.
  PDBConstructionWarning,


In [52]:
signals = ['molecular_weight', 'hydrophobicity', 'node_degree', 'node_weighted_degree', 'residue_count', 'clustering_coeff']

In [ ]:
lfc_cutoff = 0.42
model = 'weighted-rig'
print (lfc_cutoff, end=' : ')
gsp_features = pd.DataFrame(columns=signals)

for pdb in pdbinfo_dict.keys():
#         print (pdb, end=', ')
    row = []
    G = get_graph(pdbinfo_dict[pdb]['distance_matrix'], network_type=model, rig_cutoff=7.3)
    print(pdbinfo_dict[pdb]['distance_matrix'].shape)
    for signal_name in signals:
        signal = get_signal(G, pdbinfo_dict[pdb]['sequence'], signal=signal_name)
        print(len(signal))
        gftsignal = G.gft(signal)
        signal_hat = gftsignal
        value = np.sum(abs(signal_hat[G.e < G.lmax*lfc_cutoff])) / np.sum(abs(signal_hat))
        row.append(value)
    gsp_features.loc[pdb] = row

X = gsp_features

0.42 : (225, 225)
225
225
225
225
225
225
(257, 257)
257
257
257
257
257
257
(274, 274)
274
274
274
274
274
274
(239, 239)
239
239
239
239
239
239
(274, 274)
274
274
274
274
274
274
(232, 232)
232
232
232
232
232
232
(224, 224)
224
224
224
224
224
224
(272, 272)
272
272
272
272
272
272
(238, 238)
238
238
238
238
238
238
(210, 210)
210
210
210
210
210
210
(213, 213)
213
213
213
213
213
213
(251, 251)
251
251
251
251
251
251
(251, 251)
251
251
251
251
251
251
(251, 251)
251
251
251
251
251
251
(250, 250)
250
250
250
250
250
250
(205, 205)
205
205
205
205
205
205
(240, 240)
240
240
240
240
240
240
(218, 218)
218
218
218
218
218
218
(247, 247)
247
247
247
247
247
247
(337, 337)
337
337
337
337
337
337
(176, 176)
176
176
176
176
176
176
(211, 211)
211
211
211
211
211
211
(305, 305)
305
305
305
305
305
305
(257, 257)
257
257
257
257
257
257
(438, 438)
438
438
438
438
438
438
(442, 442)
442
442
442
442
442
442
(237, 237)
237
237
237
237
237
237
(297, 297)
297
297
297
297
297
297
(442, 442)
44

In [132]:
l_1 = pd.read_csv("../Protein-GSP-master/data/regression_model/Final_2Sm.csv",sep=",")
l_1 = l_1[['PDB','ln(kf)']]
l_1 = l_1.dropna()
for i in range(l_1.shape[0]):
    l_1['PDB'][i] = l_1['PDB'][i].split("(")[0]
    
for i in range(l_1.shape[0]):
    l_1['PDB'][i] = l_1['PDB'][i].split("_")[0]

l_1 = l_1.drop_duplicates(subset="PDB")


l_2 = pd.read_csv("../Protein-GSP-master/data/regression_model/Final_N2Sm.csv",sep=",")
l_2 = l_2[['PDB','ln(kf)']]
l_2 = l_2.dropna()
for i in range(l_2.shape[0]):
    l_2['PDB'][i] = l_2['PDB'][i].split("(")[0]
    
for i in range(l_2.shape[0]):
    l_2['PDB'][i] = l_2['PDB'][i].split("_")[0]

l_2 = l_2.drop_duplicates(subset="PDB")


df = pd.read_csv('../Protein-GSP-master/data/regression_model/data_all_proteins.csv')
l_3 = df[['PDB ID','Ln.K_f.']]

l_3.columns = ['PDB','ln(kf)']

final_lnkf = pd.concat([l_1,l_2,l_3],axis=0)


# to_remove = np.intersect1d(l_1['PDB'],l_2['PDB'])
# lnkf = lnkf[~lnkf['PDB'].isin(to_remove)]
# print(lnkf.shape)

# lnkf.columns = ['PDB ID','Ln.K_f.']
# final_lnkf = pd.concat([lnkfs,lnkf],axis=0)

# final_lnkf.to_csv("../Protein-GSP-master/data/regression_model/final_lnkf.csv",index=False,sep=",")

# lnkf.to_csv("../Protein-GSP-master/data/regression_model/download_new_pdb.csv",index=False,sep=",")

/home/cellsearch/py_36_env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/cellsearch/py_36_env/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/cellsearch/py_36_env/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/cellsearch/py_36_env/lib/python3.6/site-packages/ip

In [141]:
final_lnkf.groupby(final_lnkf.index).first()

,PDB,ln(kf)
0,1ARR,9.20
1,1BA5,5.90
2,1CUN,4.80
3,1AYI,5.70
4,1E41,6.90
...,...,...
84,3O4B,4.30
85,3O49,1.70
86,3O4D,4.90
87,2L6R,10.30


In [130]:
import Bio
import os
from Bio.PDB import PDBList
'''Selecting structures from PDB'''
lnkf_1 = pd.read_csv("../Protein-GSP-master/data/regression_model/download_new_pdb.csv",sep=",")
pdb1 = lnkf_1['PDB ID']
lnkf_2 = pd.read_csv("../Protein-GSP-master/data/regression_model/download_new_pdb_2.csv",sep=",")
pdb2 = lnkf_2['PDB ID']
pdbl = PDBList()
PDBlist2=list()
PDBlist2.extend(pdb1)
PDBlist2.extend(pdb2)
# print(len(PDBlist2))
for i in PDBlist2:
    i = i.strip()
    os.mkdir('../Protein-GSP-master/data/regression_model/test_pdb/'+str(i))
    pdbl.retrieve_pdb_file(i,pdir='../Protein-GSP-master/data/regression_model/test_pdb/'+str(i),file_format='pdb')